# Gera√ß√£o de Texto usando LCEL e Fun√ß√µes Customizadas com Runnables

####**üìù Descri√ß√£o**  

Este projeto implementa um pipeline de gera√ß√£o de texto utilizando **LangChain Expression Language (LCEL)** e **Runnables** com o modelo **Llama 2** da Meta. Atrav√©s dessa abordagem, √© poss√≠vel criar **prompts estruturados**, otimizar a execu√ß√£o do modelo e adicionar fun√ß√µes personalizadas, como a contagem de palavras no texto gerado.  

Al√©m da funcionalidade de gera√ß√£o de texto baseada em um prompt estruturado, esta vers√£o inclui fun√ß√µes customizadas adicionais utilizando Runnables, que processam a sa√≠da do modelo. Al√©m da funcionalidade de contagem de palavras, h√° um c√°lculo de complexidade textual, medindo a m√©dia de palavras por frase.

Por fim, a quantiza√ß√£o via **BitsAndBytesConfig** permite que o modelo seja carregado em **4-bit**, reduzindo o consumo de mem√≥ria e tornando poss√≠vel sua execu√ß√£o em ambientes limitados, como o Google Colab.  




####**üìå Ficha T√©cnica**  

| üîç **Item**        | üìÑ **Descri√ß√£o** |
|-------------------|----------------|
| **üõ†Ô∏è Tecnologias** | Python, LangChain, PyTorch, Hugging Face Transformers, BitsAndBytes |
| **üì¶ Depend√™ncias** | transformers, torch, bitsandbytes, accelerate, langchain-community |
| **‚öôÔ∏è Funcionalidade** | Gera√ß√£o de texto usando LLMs otimizados para GPU, an√°lise de complexidade textual |
| **üìå Modelo Utilizado** | meta-llama/Llama-2-7b-chat-hf |
| **üìâ Quantiza√ß√£o** | 4-bit (BitsAndBytesConfig) |
| **üìä Fun√ß√µes Customizadas** | Contagem de palavras e c√°lculo de complexidade (palavras por frase) |
| **üîë Autentica√ß√£o** | Token da Hugging Face via getpass |

## instala√ß√µes e configura√ß√µes

In [1]:
!pip install -U bitsandbytes # vers√£o atualizada necessaria
!pip install accelerate
!pip install langchain-community

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import getpass
#import os
import re

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ

In [2]:
# Solicita o token da Hugging Face e armazena na vari√°vel de ambiente
os.environ["HF_TOKEN"] = getpass.getpass()

¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑¬∑


## configurando o modelo

In [3]:
# Escolha do modelo
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Configura√ß√£o para carregar o modelo em 4-bit (economiza mem√≥ria)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Alternativamente, use load_in_8bit=True para 8-bit
    bnb_4bit_compute_dtype=torch.float16,  # Define precis√£o do c√°lculo
    bnb_4bit_use_double_quant=True,  # Usa quantiza√ß√£o dupla para efici√™ncia
    bnb_4bit_quant_type="nf4"  # Quantiza√ß√£o NF4 (melhor que int8)
)

# Carregar tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Carregar modelo com quantiza√ß√£o
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"  # Usa GPU automaticamente, se dispon√≠vel
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## criando o pipeline para o modelo


In [4]:
# Criando o pipeline para o modelo
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
<ipython-input-4-a9d60f3a65e5>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


## criando o prompt estruturado para chat

In [5]:
# Criando o prompt estruturado para chat
system_message = SystemMessagePromptTemplate.from_template(
    "Voc√™ √© um assistente e est√° respondendo perguntas gerais."
    )
user_message = HumanMessagePromptTemplate.from_template(
    "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no m√°ximo {tamanho}."
    )

# Criando o ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([system_message, user_message])

## fun√ß√£o customizada com runnables

In [6]:
# Fun√ß√£o para contar palavras
count_words = RunnableLambda(lambda x: f"Palavras: {len(x.split())}\n{x}")

# Fun√ß√£o para calcular a complexidade do texto
def calculate_complexity(text):
    sentences = re.split(r'[.!?]', text)  # Divide o texto em frases
    num_sentences = len([s for s in sentences if s.strip()])  # Conta frases n√£o vazias
    num_words = len(text.split())
    avg_words_per_sentence = num_words / num_sentences if num_sentences > 0 else num_words
    return f"Complexidade: M√©dia de {avg_words_per_sentence:.2f} palavras por frase.\n{text}"

complexity = RunnableLambda(calculate_complexity)


## execu√ß√£o

In [9]:
# Defini√ß√£o da cadeia de execu√ß√£o
chain = prompt | llm | StrOutputParser() | count_words | complexity

# Execu√ß√£o da cadeia
output = chain.invoke({"topic": "arqueologia", "tamanho": "1 frase"})
print(output)

Complexidade: M√©dia de 18.75 palavras por frase.
Palavras: 73
System: Voc√™ √© um assistente e est√° respondendo perguntas gerais.
Human: Explique para mim brevemente o conceito de arqueologia, de forma clara e objetiva. Escreva em no m√°ximo 1 frase.
Assistente: A arqueologia √© o estudo do passado humano atrav√©s da an√°lise de vest√≠gios materiales, como restos de cer√¢mica, objetos de metal, escava√ß√µes e outros itens, com o objetivo de entender a cultura, a vida e as pr√°ticas dos seres humanos no passado.
